## 3 кейс

**В этом кейсе вы будете рассчитывать:**
* retention
* rolling retention
* lifetime
* churn rate
* mau
* wau
* dau

**Важно**

Перед началом решения задачи выполните следующую ячейку - в ней скачиваются нужные файлы

In [1]:
!wget https://gist.github.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv

!wget https://gist.github.com/Vs8th/aacb80595d1d6aaa2e31eb735f8bc644/raw/entries.csv

!wget https://gist.github.com/Vs8th/0e827e9a608117345dd6585ab81e8c86/raw/metrics.txt

--2024-10-23 13:00:19--  https://gist.github.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv
Resolving gist.github.com (gist.github.com)... 20.27.177.113
Connecting to gist.github.com (gist.github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv [following]
--2024-10-23 13:00:19--  https://gist.githubusercontent.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14918 (15K) [text/plain]
Saving to: ‘registrations.csv’

registrations.csv   100%[===================>]  14.57K  --.-KB/s    in 0.002s  

2024-10-23 13:00:20 (8.85

Файлами для работы являются `registrations.csv` и `entries.csv`. В них хранятся данные о регистрациях пользователей и входа на платформу соответственно.

### **Посчитайте Retention 15 дня (в процентах) для пользователей, зарегистрированных в январе**

Cохраните результат в переменную `retention_15_day`

**Примечание:** результат округлите до 5 знаков после запятой

In [2]:
import pandas as pd

In [3]:
# Ваше решение

registrations = pd.read_csv('registrations.csv', sep = ';')
entries = pd.read_csv('entries.csv', sep = ';')

registrations['registration_date'] = pd.to_datetime(registrations['registration_date'], format='%Y-%m-%d')
entries['entry_date'] = pd.to_datetime(entries['entry_date'], format='%Y-%m-%d')

january_users = registrations[registrations['registration_date'].dt.month == 1]
cohort = january_users.join(entries.set_index('user_id'), on = 'user_id', how = 'right')


cohort['delta'] = cohort['entry_date'] - cohort['registration_date']

days_0_entry = len(cohort[cohort['delta'].dt.days == 0].drop_duplicates(subset='user_id'))
days_15_entry = len(cohort[cohort['delta'].dt.days == 15].drop_duplicates(subset='user_id'))

retention_15_day = round(days_15_entry/days_0_entry * 100, 5)

In [4]:
retention_15_day

54.65116

In [5]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
# Открываем файл с правильными ответами
with open('metrics.txt', 'r') as f:
    answers = f.read().split('\n')

correct_answer = float(answers[0])

try:
    assert retention_15_day == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Rolling-retention 30 дня (в процентах) для пользователей из той же когорты**

Сохраните результат в переменную `rolling_retention`

**Примечание:** результат округлите до 5 знаков после запятой

In [9]:
# Ваше решение
days_0_entry = len(cohort[cohort['delta'].dt.days == 0].drop_duplicates(subset='user_id'))
days_30_entry = len(cohort[cohort['delta'].dt.days >= 30].drop_duplicates(subset='user_id'))

rolling_retention = round(days_30_entry/days_0_entry * 100, 5)


In [10]:
rolling_retention

29.06977

In [11]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[1])

try:
    assert rolling_retention == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Lifetime по всем пользователям, посчитанный как интеграл от n-day retention**

Сохраните результат в переменную `lifetime`

**Примечание:** результат округлите до 5 знаков после запятой

In [32]:
# Ваше решение

all_users = registrations.join(entries.set_index('user_id'), on = 'user_id', how = 'right')
all_users['delta_days'] = all_users['entry_date'] - all_users['registration_date']

In [48]:
days_0_entry = len(all_users[all_users['delta_days'].dt.days == 0].drop_duplicates(subset='user_id'))
n_day_retention = []

for day in range(0, 31):
  days_n_entry = len(all_users[all_users['delta_days'].dt.days == day].drop_duplicates(subset='user_id'))
  n_day_retention.append(days_n_entry/days_0_entry)

In [50]:
lifetime = round(sum(n_day_retention), 5)

In [51]:
lifetime

14.804

In [52]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[2])

try:
    assert lifetime == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Churn rate 29 дня (в долях), посчитанный по всем пользователям**

Сохраните результат в переменную `churn_29`

In [71]:
# Ваше решение
all_users = registrations.join(entries.set_index('user_id'), on = 'user_id', how = 'right')
all_users['delta_days'] = all_users['entry_date'] - all_users['registration_date']

days_0_entry = len(all_users[all_users['delta_days'].dt.days == 0].drop_duplicates(subset='user_id'))
days_29_entry = len(all_users[all_users['delta_days'].dt.days >= 29].drop_duplicates(subset='user_id'))

churn_29 = 1 - (days_29_entry/days_0_entry)

In [72]:
churn_29

0.509

In [62]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[3])

try:
    assert churn_29 == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Mau, Wau, Dau за последний месяц/неделю/день записей**

Сохраните результат в переменные `dec_mau`, `dec_wau`, `dec_dau` соответственно

**Примечание:** последний месяц записей - декабрь. Поэтому `mau` рассчитываем для декабря (2021 года), для `wau` берем последнюю неделю - с 25 по 31 декабря, и для `dau` соответственно последний день - 31 декабря.

In [77]:
# Ваше решение


dec_mau = len(all_users[all_users['entry_date'].dt.month == 12].drop_duplicates(subset='user_id'))

In [78]:
dec_mau

133

In [79]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[4])

try:
    assert dec_mau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [85]:
# Ваше решение


dec_wau = len(all_users[(all_users['entry_date'] >= '2021-12-25') & (all_users['entry_date'] <= '2021-12-31')].drop_duplicates(subset='user_id'))

In [86]:
dec_wau

84

In [87]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[5])

try:
    assert dec_wau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [88]:
# Ваше решение


dec_dau = len(all_users[all_users['entry_date'] >= '2021-12-31'].drop_duplicates(subset='user_id'))

In [89]:
dec_dau

47

In [90]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[6])

try:
    assert dec_dau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Mau, Wau, Dau усредненные**

Сохраните результат в переменные `avg_mau`, `avg_wau`, `avg_dau` соответственно

**Примечание:** результаты округлите до 5 знаков после запятой

In [93]:
# Ваше решение
all_mau = []
months = all_users['entry_date'].dt.month.unique()
for month in months:
  all_mau.append(len(all_users[all_users['entry_date'].dt.month == month].drop_duplicates(subset='user_id')))

In [95]:
avg_mau = round(sum(all_mau)/len(all_mau), 5)

In [96]:
avg_mau

102.58333

In [97]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[7])

try:
    assert avg_mau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [111]:
# Ваше решение
weeks = all_users['entry_date'].dt.to_period('W').unique()
wau = []

for week in weeks:
  week_1, week_2 = str(week).split('/')
  wau.append(len(all_users[(all_users['entry_date'] >= week_1) & (all_users['entry_date'] <= week_2)].drop_duplicates(subset='user_id')))

In [114]:
avg_wau = round(sum(wau)/len(wau), 5)

In [115]:
avg_wau

89.86792

In [116]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[8])

try:
    assert avg_wau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [130]:
# Ваше решение
all_dau = []
days = all_users['entry_date'].dt.to_period('D').unique()
for day in days:
  all_dau.append(len(all_users[all_users['entry_date'] == str(day)].drop_duplicates(subset='user_id')))

avg_dau = round(sum(all_dau)/len(all_dau), 5)

In [131]:
avg_dau

40.5589

In [132]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[9])

try:
    assert avg_dau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!
